### Introduction

The objective of this example is to run the basic calibration steps on raw frames.
By the end of this example, we will know how to:

- Pre-process raw frames to remove the overscan section and extract the science section
- Remove cosmic rays from individual frames using the L.A. algorithm (Van Dokkum+2005)
- Produce calibrated bias, dark and pixelflat frames

Let's start with some imports and function definitions

In [5]:
import os

from lvmdrp.core.constants import LVM_UNAM_URL
from lvmdrp.utils.examples import GAINS, RDNOISES
from lvmdrp.utils.examples import fetch_example_data, get_frames_metadata

from lvmdrp.functions import imageMethod

In [6]:
print(LVM_UNAM_URL)

http://ifs.astroscu.unam.mx/LVM


### Downloading 2D simulations

Before we begin, we need to:
- Define paths to input and output data
- Download some data to run through the pipeline

In this case, we'll download 2D simulated data.

In [1]:
from tree import Tree
tree = Tree('sdss5')

In [5]:
from sdss_access.path import Path
path = Path(release='sdss5')
path.full('lvm_lab', hemi='s', camspec='b1', mjd='59580', expnum='1')

'/Users/Brian/Work/sdss/sas/sdsswork/data/lvm/lab/59580/sdR-s-b1-00000001.fits.gz'

In [10]:
# define input data directory
#data_path = os.path.abspath(os.path.join("..", "data"))
data_path = os.getenv("LVM_DATA_S")
os.makedirs(data_path, exist_ok=True)
print(data_path)

# let's create the output directory
#output_path = "./data"
output_path = os.getenv("LVM_SPECTRO_REDUX")
os.makedirs(output_path, exist_ok=True)
print(output_path)

# create processed frames path template
out_path = os.path.join(output_path, "lvm-{kind}-{camera}-{expnum}.{ext}")
mpixelfiber_path = os.path.join(output_path, "lvm-mpixelflat-{camera}.{ext}")

print(out_path)
print(mpixelfiber_path)

# now we download the example data
#fetch_example_data(url=LVM_UNAM_URL, name="sim_2d", dest_path=data_path)

/Users/Brian/Work/sdss/sas/sdsswork/data/lvm/lco
/Users/Brian/Work/sdss/sas/sdsswork/lvm/spectro/redux
/Users/Brian/Work/sdss/sas/sdsswork/lvm/spectro/redux/lvm-{kind}-{camera}-{expnum}.{ext}
/Users/Brian/Work/sdss/sas/sdsswork/lvm/spectro/redux/lvm-mpixelflat-{camera}.{ext}


In order to organize our simulated data and target specific type of frames better,
we extract relevant metadata from the frames and group them by image type, camera
and exposure time.

In [11]:
# extract metadata
frames_table = get_frames_metadata(path=os.path.join(data_path, "sim_2d"), ignore_cache=True)
# group by imagetyp, camera and exptime
frames_table = frames_table.group_by(["imagetyp", "spec", "camera", "expnum", "exptime"])
# define groups and unique categories in those groups
frames_groups = frames_table.groups
unique_groups = frames_groups.keys

[INFO]: extracting metadata from 0 frames


0it [00:00, ?it/s]

[INFO]: successfully extracted metadata


IndexError: index 0 is out of bounds for axis 0 with size 0

We will start by processing calibration frames: bias, dark and pixelflats

Let's start with the biases for the blue channel of spectrograph #1...

In [4]:
# OK, biases have exptime != 0 for some reason...
cameras = ["b", "r", "z"]
spec = 1
for camera in cameras:
    biases = frames_groups[(unique_groups["imagetyp"]=="bias")&(unique_groups["spec"]==f"sp{spec}")&(unique_groups["camera"]==f"{camera}{spec}")]
    print(biases)

    # let's pre-process the all bias frames
    pbias_paths = []
    for bias_frame in biases:
        bias_path = bias_frame["path"]
        bias_camera, bias_expnum = bias_frame["camera"], bias_frame["expnum"]

        pbias_path = out_path.format(kind="pbias", camera=bias_camera, expnum=bias_expnum, ext="fits")

        gains = ",".join(map(str, GAINS[camera]))
        rdnoises = ",".join(map(str, RDNOISES[camera]))
        imageMethod.preprocRawFrame_drp(
            in_image=bias_path,
            out_image=pbias_path,
            subtract_overscan=False, unit="e-", compute_error=True,
            assume_imagetyp="flatfield", assume_gain=gains, assume_rdnoise=rdnoises
        )
        pbias_paths.append(pbias_path)

    mpixelflat_path = os.path.join(output_path, f"lvm-mpixelflat-{bias_camera}.fits")
    imageMethod.createMasterFrame_drp(
        in_images=pbias_paths,
        out_image=mpixelflat_path.format(camera=bias_camera, ext="fits"),
    )
    imageMethod.basicCalibration_drp(
        in_image=mpixelflat_path.format(camera=bias_camera, ext="fits"),
        out_image=mpixelflat_path.format(camera=bias_camera, ext="fits")
    )

imagetyp spec camera  expnum  exptime                                          path                                        
-------- ---- ------ -------- ------- -------------------------------------------------------------------------------------
    bias  sp1     b1 00000101     0.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-b1-00000101.fits.gz
    bias  sp1     b1 00000102     0.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-b1-00000102.fits.gz
    bias  sp1     b1 00000103     0.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-b1-00000103.fits.gz
    bias  sp1     b1 00000104     0.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-b1-00000104.fits.gz
    bias  sp1     b1 00000105     0.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-b1-00000105.fits.gz
    bias  sp1     b1 00000106     0.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-b1-00000106.fits.gz
    bias

In [5]:
fiberflats = frames_groups[(unique_groups["imagetyp"]=="flat")&(unique_groups["spec"]=="sp1")&(unique_groups["expnum"]=="00001002")]
print(fiberflats)

# Now that we have calibration frames, let's calibrate a fiberflat and an arc
for fiberflat_frame in fiberflats:
    fiberflat_path = fiberflat_frame["path"]
    fiberflat_camera, fiberflat_expnum = fiberflat_frame["camera"], fiberflat_frame["expnum"]
    
    gains = ",".join(map(str, GAINS[camera]))
    rdnoises = ",".join(map(str, RDNOISES[camera]))
    imageMethod.preprocRawFrame_drp(
        in_image=fiberflat_path,
        out_image=out_path.format(kind="pfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        subtract_overscan=False, unit="e-", compute_error=True,
        assume_imagetyp="fiberflat", assume_gain=gains, assume_rdnoise=rdnoises
    )
    # imageMethod.LACosmic_drp(
    #     in_image=out_path.format(kind="pfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
    #     out_image=out_path.format(kind="rfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
    #     iter=1, increase_radius=1, flim=1.3, parallel="auto"
    # )
    imageMethod.basicCalibration_drp(
        in_image=out_path.format(kind="pfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        out_image=out_path.format(kind="cfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        in_bias=out_path.format(kind="pbias", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        in_pixelflat=mpixelflat_path.format(camera=fiberflat_camera, ext="fits")
    )

imagetyp spec camera  expnum  exptime                                          path                                        
-------- ---- ------ -------- ------- -------------------------------------------------------------------------------------
    flat  sp1     b1 00001002    10.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-b1-00001002.fits.gz
    flat  sp1     r1 00001002    10.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-r1-00001002.fits.gz
    flat  sp1     z1 00001002    10.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-z1-00001002.fits.gz
[WARNING]: divide by zero encountered in divide (RuntimeWarning)


In [6]:
arcs = frames_groups[(unique_groups["imagetyp"]=="arc")&(unique_groups["spec"]=="sp1")&(unique_groups["expnum"]=="00010001")]
print(arcs)

for arc_frame in arcs:
    arc_path = arc_frame["path"]
    arc_camera, arc_expnum = arc_frame["camera"], arc_frame["expnum"]
    
    gains = ",".join(map(str, GAINS[camera]))
    rdnoises = ",".join(map(str, RDNOISES[camera]))
    imageMethod.preprocRawFrame_drp(
        in_image=arc_path,
        out_image=out_path.format(kind="parc", camera=arc_camera, expnum=arc_expnum, ext="fits"),
        subtract_overscan=False, unit="e-", compute_error=True,
        assume_imagetyp="arc", assume_gain=gains, assume_rdnoise=rdnoises
    )
    # imageMethod.LACosmic_drp(
    #     in_image=out_path.format(kind="parc", camera=arc_camera, expnum=arc_expnum, ext="fits"),
    #     out_image=out_path.format(kind="rarc", camera=arc_camera, expnum=arc_expnum, ext="fits"),
    #     iter=3, increase_radius=1, flim=1.3, parallel="auto"
    # )
    imageMethod.basicCalibration_drp(
        in_image=out_path.format(kind="parc", camera=arc_camera, expnum=arc_expnum, ext="fits"),
        out_image=out_path.format(kind="carc", camera=arc_camera, expnum=arc_expnum, ext="fits"),
        in_bias=out_path.format(kind="pbias", camera=arc_camera, expnum=arc_expnum, ext="fits"),
        in_pixelflat=mpixelflat_path.format(camera=arc_camera, ext="fits")
    )

imagetyp spec camera  expnum  exptime                                          path                                        
-------- ---- ------ -------- ------- -------------------------------------------------------------------------------------
     arc  sp1     b1 00010001    10.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-b1-00010001.fits.gz
     arc  sp1     r1 00010001    10.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-r1-00010001.fits.gz
     arc  sp1     z1 00010001    10.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-z1-00010001.fits.gz
